An Introduction to interacting with nifti volumes in python.
Goals
Demonstrate data input and output with nibabel.
Demonstrate interacting with nifi headers.
Demonstrate manipulating image volumes.
Nibabel and Nilearn are two very useful packages for neuroimaging data manipulation.

Nibabel is a package designed for interacting with neuroimaging data in common formats.

Nilearn is a much broader package that has the functionality to perform machine learning. There are examples of all the wonderful things you can do here. However, I will not be covering machine learning here. I will only be talking about some of the useful functions that nilearn has for plotting/manipulating data.

Data input and output.
First, let's set the path to where our data are located. If you took part in the nipype tutorial, then this is the same data we used before.conda 

In [89]:
bidspath='/Users/Carolyn/Desktop/NeuroHack/'
#atlaspath='/opt/neurohack/examp/maxprob_vol_rh.nii.gz'


Now set the path to the outputs. You will need to change my username for yours.

In [90]:
outputpath='/Users/Carolyn/Desktop/NeuroHack/output/'

To help us interpret what comes below, I am going to plot the data.

In [91]:
import os.path as op
import os
from glob import glob
import pandas as pd
import nibabel as nib

In [124]:
import nilearn
from nilearn import image
from nilearn import plotting
import matplotlib.pyplot as plt
import numpy as np

In [120]:
class bids:
    
    # Initialises by generating a list of subjects/sessions/tasks. Requires only the base path to the BIDS file. 
    def __init__(self,path):
        self.path=path
        self.subjects=[op.basename(s).split('-')[1] for s in
            sorted(glob(op.join(self.path, 'sub-*'))) if op.isdir(s)]
        
        self.submessage=f"Found {len(self.subjects)} participant(s) {self.subjects}"
        
        sessions = []
        sessmessage=[]
        for this_sub in self.subjects:
            these_ses = [
                op.basename(s).split('-')[1] for s in
                sorted(
                    glob(op.join(self.path, f'sub-{this_sub}', 'ses-*')))
                if op.isdir(s)
            ]
            
            sessmessage.append(f"Found {len(these_ses)} session(s) for sub-{this_sub} {these_ses}")
            sessions.append(these_ses)
            
        self.sessions=sessions
        self.sessmessage=sessmessage
        
        tasks=[]
        taskmessage=[]
        for this_sub, these_ses in zip(self.subjects, self.sessions):
            these_task = []
            for this_ses in these_ses:
                if this_ses is None:  
                    tmp = glob(op.join(
                        self.path,
                        f'sub-{this_sub}',
                        'func',
                        f"*{'.nii'}*"
                    ))
                else:
                    tmp = glob(op.join(
                        self.path,
                        f'sub-{this_sub}',
                        f'ses-{this_ses}',
                        'func',
                        f"*{'.nii'}*"
                    ))

                these_ses_task = list(set(
                    [op.basename(f).split('task-')[1].split('_')[0]
                     for f in tmp]
                ))
                
     
                nullstring = "" if this_ses is None else f"and ses-{this_ses}"

                taskmessage.append(f"Found {len(these_ses_task)} task(s) for sub-{this_sub} {nullstring} {these_ses_task}")
                these_task.append(these_ses_task)
                


            self.taskmessage=taskmessage
            tasks.append(these_task)
        self.tasks=tasks
        
        sessions = []
        for this_sub in self.subjects:
            these_ses = [
                op.basename(s).split('-')[1] for s in
                sorted(
                    glob(op.join(self.path, f'sub-{this_sub}', 'ses-*')))
                if op.isdir(s)
            ]
    
    # Prints everything out.
    def elaborate(self):
        print(self.submessage)
        print(self.sessmessage)
        print(self.taskmessage)
    
    # Finds a func file.
    def find_funcs(self,sub,ses,task):
    
        ffunc_dir = op.join(self.path, f'sub-{sub}', f'ses-{ses}', 'func')
        
        funcs = sorted(glob(op.join(ffunc_dir, f'*task-{task}*')))
        if not funcs:
            raise ValueError(
                "Could not find functional data with the following parameters:\n"
                f"sub-{sub}, ses-{ses}, task-{task}")
        return funcs
    
    # Finds an anatomical file.
    def find_anats(self,sub,ses,weight):
        
        anat_dir = op.join(self.path, f'sub-{sub}', f'ses-{ses}', 'anat')
        
        anats = sorted(glob(op.join(anat_dir, f'*{weight}*')))
        
        if not anats:
            raise ValueError(
                "Could not find anatomical data with the following parameters:\n"
                f"sub-{sub}, ses-{ses}, weight-{weight}")
        
        return anats
    
    # Shows a file.
    def show(self,npath):
        
        niftifiledim=len(image.load_img(npath).shape)
        if niftifiledim == 3:
           display=plotting.view_img(npath)
        else:
            print ('>1 volume, plotting only the first for convenience')
            firstim=image.index_img(npath, 0)
            display=plotting.view_img(firstim)
        return display 
        
    
    
    def getniftibits(self,npath):
        nifti = nib.load(npath)
        VOXSIZE = nifti.header['pixdim'][1:4]
        SHAPE= (nifti.header['dim'][1:5])
        TR = (nifti.header['pixdim'][4:5])
        VOXFRAME=pd.DataFrame(VOXSIZE)
        VOXFRAME=VOXFRAME.T
        SHAPEFRAME=pd.DataFrame(SHAPE)
        SHAPEFRAME=SHAPEFRAME.T
        VOXFRAME.columns=['VoxsizeX','VoxsizeY','VoxsizeZ']
        SHAPEFRAME.columns=['ShapeX','ShapeY','ShapeZ','Volumes']
        CFRAMEi=pd.concat([VOXFRAME,SHAPEFRAME],axis=1)
        CFRAMEi['TR'] = TR 
        return(CFRAMEi)

In [121]:
mybids=bids(bidspath)
inputpath

'/Macintosh HD/Users/Carolyn/Desktop/NeuroHack/'

In [122]:
mybids.elaborate()

Found 25 participant(s) ['0148', '0178', '0179', '0182', '0184', '0185', '0186', '0187', '0190', '0192', '0193', '0194', '0196', '3080', '3081', '3091', '3120', '3128', '3130', '3142', '3146', '3148', '3217', '3227', '3329']
["Found 1 session(s) for sub-0148 ['01']", "Found 2 session(s) for sub-0178 ['01', '02']", "Found 1 session(s) for sub-0179 ['01']", "Found 1 session(s) for sub-0182 ['01']", "Found 1 session(s) for sub-0184 ['01']", "Found 1 session(s) for sub-0185 ['01']", "Found 1 session(s) for sub-0186 ['01']", "Found 1 session(s) for sub-0187 ['01']", "Found 1 session(s) for sub-0190 ['01']", "Found 1 session(s) for sub-0192 ['01']", "Found 1 session(s) for sub-0193 ['01']", "Found 1 session(s) for sub-0194 ['01']", "Found 1 session(s) for sub-0196 ['01']", "Found 1 session(s) for sub-3080 ['01']", "Found 1 session(s) for sub-3081 ['01']", "Found 1 session(s) for sub-3091 ['01']", "Found 1 session(s) for sub-3120 ['01']", "Found 1 session(s) for sub-3128 ['01']", "Found 1 ses

In [127]:
pd.DataFrame(mybids.tasks)#

,0,1
0,"[infant, affect]",None
1,[],"[affect, infant]"
2,"[infant, affect]",None
3,"[infant, affect]",None
4,"[infant, affect]",None
5,"[infant, affect]",None
6,"[infant, affect]",None
7,"[infant, affect]",None
8,"[infant, affect]",None
9,"[infant, affect]",None


In [114]:
mybids.find_funcs(mybids.subjects[23],'01','affect')

['/Users/Carolyn/Desktop/NeuroHack/sub-3227/ses-01/func/sub-3227_ses-01_task-affect_run-01_bold.json',
 '/Users/Carolyn/Desktop/NeuroHack/sub-3227/ses-01/func/sub-3227_ses-01_task-affect_run-01_events.tsv',
 '/Users/Carolyn/Desktop/NeuroHack/sub-3227/ses-01/func/sub-3227_ses-01_task-affect_run-02_bold.json',
 '/Users/Carolyn/Desktop/NeuroHack/sub-3227/ses-01/func/sub-3227_ses-01_task-affect_run-02_events.tsv']